<a href="https://colab.research.google.com/github/rmccormick314/GFSAD_Crop_Type_Mapping/blob/main/AppEARS_Data_Selector.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
### 0. Initialize Google Earth Engine API

# Import the Earth Engine package.
import ee
import geemap

# Trigger the authentication flow.
ee.Authenticate()

# Initialize the library.
ee.Initialize()

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://code.earthengine.google.com/client-auth?scopes=https%3A//www.googleapis.com/auth/earthengine%20https%3A//www.googleapis.com/auth/devstorage.full_control&request_id=WKHsG8ZhupYVPULIMi8vfK6eOLoYJUzv_JL1-ypXSdo&tc=CVvFQgl0gOBq85Ra2DlQfIj2OC3cOEyVWp7keRHtq0g&cc=PYydJlZVEgMP0g1Bd52keThsR3Hqudtg0cA0QIodFuw

The authorization workflow will generate a code, which you should paste in the box below.
Enter verification code: 4/1AfJohXnlsBbxTOTmKx8Q3WTswyZ-WepliNgKnaAVmYqtY63KgOyE-csiowg

Successfully saved authorization token.


In [2]:
# 1a. Set Up the Working Environment

#### Import the required packages and set the input/working directory.
import requests as r
import getpass, pprint, time, os, cgi

#### Set input directory, change working directory
inDir = "./" # Input directory needs to be set proper if running this in a real environment.
os.chdir(inDir)
api = 'https://appeears.earthdatacloud.nasa.gov/api/'

#### Request username and password from user.
user = getpass.getpass(prompt = 'Enter NASA Earthdata Login Username: ')
password = getpass.getpass(prompt = 'Enter NASA Earthdata Login Password: ')

#### Call the log-in service with credentials, acquire bearer token,
#### and delete the user information once request is complete.
token_response = r.post('{}login'.format(api), auth=(user, password)).json()
del user, password

token_response # Token is good for 48 hours

Enter NASA Earthdata Login Username: ··········
Enter NASA Earthdata Login Password: ··········


{'token_type': 'Bearer',
 'token': 'UPLY0wFANsbWs6eqoD1lJQCpsK6Wb476g1UARdLSXGoM8aPmZMU9RLUg89eYxptE1TfkRUPH37YN3dPJYkTP-A',
 'expiration': '2023-09-27T22:56:13Z'}

In [5]:
# 2. Query Available Products
# 2a. Search and Explore Available Products

product_response = r.get('{}product'.format(api)).json()                         # request all products in the product service
print('AρρEEARS currently supports {} products.'.format(len(product_response)))

#### Next, create a dictionary indexed by product name, making it easier to query a specific product.
products = {p['ProductAndVersion']: p for p in product_response} # Create a dictionary indexed by product name & version
prods = ['HLSS30.020']

lst_response = r.get('{}product/{}'.format(api, prods[0])).json()  # Request layers for the 1st product (index 0) in the list
pprint.pprint(products['HLSS30.020'])

AρρEEARS currently supports 120 products.
{'Available': True,
 'DOI': '10.5067/HLS/HLSS30.002',
 'Deleted': False,
 'Description': 'Land Surface Reflectance',
 'DocLink': 'https://doi.org/10.5067/HLS/HLSS30.002',
 'Info': {'cmr_date_parse': 'date',
          'extension': 'tif',
          'is_utm_zonal': True,
          'provider_id': 'lp_prod'},
 'Platform': 'Harmonized Landsat and Sentinel-2',
 'Product': 'HLSS30',
 'ProductAndVersion': 'HLSS30.020',
 'RasterType': 'Tile',
 'Resolution': '30m',
 'Source': 'LP DAAC',
 'TemporalExtentEnd': 'Present',
 'TemporalExtentStart': '2015-12-01',
 'TemporalGranularity': 'Daily',
 'Version': '020'}


In [9]:
lai_response = r.get('{}product/{}'.format(api, prods[0])).json()  # Request layers for the 1st product (index 0) in the list: MCD15A3H.006
#list(lai_response.keys())                                          # Print the LAI layer names

layers = []

for key in (list(lai_response.keys())):
  layers.append((prods[0],key)) # Append to tupled list linking desired product with desired layers

prodLayer = []
for l in layers:
    prodLayer.append({
            "layer": l[1],
            "product": l[0]
          })
prodLayer

[{'layer': 'B01', 'product': 'HLSS30.020'},
 {'layer': 'B02', 'product': 'HLSS30.020'},
 {'layer': 'B03', 'product': 'HLSS30.020'},
 {'layer': 'B04', 'product': 'HLSS30.020'},
 {'layer': 'B05', 'product': 'HLSS30.020'},
 {'layer': 'B06', 'product': 'HLSS30.020'},
 {'layer': 'B07', 'product': 'HLSS30.020'},
 {'layer': 'B08', 'product': 'HLSS30.020'},
 {'layer': 'B09', 'product': 'HLSS30.020'},
 {'layer': 'B10', 'product': 'HLSS30.020'},
 {'layer': 'B11', 'product': 'HLSS30.020'},
 {'layer': 'B12', 'product': 'HLSS30.020'},
 {'layer': 'B8A', 'product': 'HLSS30.020'},
 {'layer': 'Fmask', 'product': 'HLSS30.020'},
 {'layer': 'SAA', 'product': 'HLSS30.020'},
 {'layer': 'SZA', 'product': 'HLSS30.020'},
 {'layer': 'VAA', 'product': 'HLSS30.020'},
 {'layer': 'VZA', 'product': 'HLSS30.020'}]

In [58]:
# 3. Submit an Area Request
token = token_response['token']                      # Save login token to a variable
head = {'Authorization': 'Bearer {}'.format(token)}  # Create a header to store token information, needed to submit a request

# import shapefile
gaez6 = ee.FeatureCollection("projects/usgs-gee-wgscflag/assets/GFSAD30-RAEZ-74-Zones")
gaez6 = gaez6.filter( ee.Filter.eq( 'Zone', '6' ) )

states = ee.FeatureCollection("TIGER/2018/States");
south_dakota = states.filter(ee.Filter.eq('NAME', 'South Dakota')).first()
new_geom = ee.FeatureCollection(gaez6.geometry().intersection(south_dakota.geometry()))

# build map, add shapefile, center on shapefile
Map = geemap.Map()
Map.addLayer(gaez6)
Map.centerObject(gaez6)

import geopandas
import json

gdf = geemap.ee_to_geopandas( new_geom ).to_json()
gdf = json.loads(gdf)
# display map
Map

Map(center=[20, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(Togg…

In [59]:
projections = r.get('{}spatial/proj'.format(api)).json()  # Call to spatial API, return projs as json
# projections                                               # Print projections and information

projs = {}                                  # Create an empty dictionary
for p in projections: projs[p['Name']] = p  # Fill dictionary with `Name` as keys
#list(projs.keys())                          # Print dictionary keys

print(projs['geographic']['Name'])

# 3c. Compile a JSON [Task Object]
task_name = input('Enter a Task Name: ') # User-defined name of the task: 'NPS Vegetation Area' used in example

task_type = ['point','area']        # Type of task, area or point
proj = projs['geographic']['Name']  # Set output projection
outFormat = ['geotiff', 'netcdf4']  # Set output file format type
startDate = '07-01-2020'            # Start of the date range for which to extract data: MM-DD-YYYY
endDate = '07-31-2020'              # End of the date range for which to extract data: MM-DD-YYYY
recurring = False                   # Specify True for a recurring date range
#yearRange = [2000,2016]            # if recurring = True, set yearRange, change start/end date to MM-DD

task = {
    'task_type': task_type[1],
    'task_name': task_name,
    'params': {
         'dates': [
         {
             'startDate': startDate,
             'endDate': endDate
         }],
         'layers': prodLayer,
         'output': {
                 'format': {
                         'type': outFormat[0]},
                         'projection': proj},
         'geo': gdf,
    }
}

task_response = r.post('{}task'.format(api), json=task, headers=head).json()  # Post json to the API task service, return response as json
task_response                                                                 # Print task response

geographic
Enter a Task Name: South Dakota TEST


{'task_id': 'c7c49add-dc03-4b75-855e-977344f0d479', 'status': 'pending'}

In [ ]:
task_id = task_response['task_id']                                               # Set task id from request submission
status_response = r.get('{}status/{}'.format(api, task_id), headers=head).json() # Call status service with specific task ID & user credentials
#status_response                                                                                                                      # Print task response

# Ping API until request is complete, then continue to Section 4
starttime = time.time()
while r.get('{}task/{}'.format(api, task_id), headers=head).json()['status'] != 'done':
    print(r.get('{}task/{}'.format(api, task_id), headers=head).json()['status'])
    time.sleep(20.0 - ((time.time() - starttime) % 20.0))
print(r.get('{}task/{}'.format(api, task_id), headers=head).json()['status'])

processing
processing
processing
processing
processing
processing
processing
processing
processing
processing
processing
processing
processing
processing
processing
processing
processing
processing
processing
processing
processing
processing
processing
processing
processing
processing
processing
processing
processing
processing
processing
processing
processing
processing
processing
processing
processing
processing
processing
processing
processing
processing
processing
processing
processing
processing
processing
processing
processing
processing
processing
processing
processing
processing
processing
processing
processing
processing
processing
processing
processing
processing
processing
processing
processing
processing
processing
processing
processing
processing
processing
processing
processing
processing
processing
processing
processing
processing
processing
processing
processing
processing
processing
processing
processing
processing
processing
processing
processing
processing
processing